<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий на hh.ru
   

In [ ]:
clients = [8, 4, 9, 6, 7, 1, 6, 8, 9, 10]
def CSI_sort(clients):
    new_list = sorted(clients)
    max_grade = []
    for elem in new_list:
        if elem not in max_grade:
            max_grade.append(elem)
    return max_grade[-3:]
CSI_sort(clients)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
#ваш код здесь
hh_data = pd.read_csv('/home/gor/Загрузки/data_for_ds/dst-3.0_16_1_hh_database.csv', sep = ';')
data = hh_data.copy()

In [ ]:
data['Город, переезд, командировки']

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
data.head(2)

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
#ваш код здесь
data.info()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
#ваш код здесь#ваш код здесь
data.describe()

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
#ваш код здесь
def education_level(education):
    ed_list = ['высшее','неоконченное высшее','среднее специальное','среднее']
    check_str = education.lower().split()[0] + " " + education.lower().split()[1]
    
    if check_str in ed_list:
        return check_str
    
    elif education.lower().split()[0] in ed_list:
        return education.lower().split()[0]

data['Образование'] = data['Образование и ВУЗ'].apply(education_level)
data[data['Образование'] == 'среднее'].shape[0]

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
#ваш код здесь
def sex_detection(inform):
    sex = inform.split(sep = ',')[0].strip(' ')
    if sex == 'Женщина': return 'Ж'
    elif sex == 'Мужчина': return 'М'
    else: return 'Unknown'
    
def age_detection(inform):
    return int(inform.split(sep = ',')[1].split()[0].strip(' '))

data['Пол'] = data['Пол, возраст'].apply(sex_detection)
data['Возраст'] = data['Пол, возраст'].apply(age_detection)
data = data.drop(['Пол, возраст'], axis = 1)   

In [ ]:
is_null = data.isnull()
is_null[is_null['Опыт работы']==True]['Опыт работы']

3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
#ваш код здесь
def experience_detection(information):
    if information == '': return None
    elif information is None: return None
    elif information is np.nan: return None
    else:
        int_list= []
        years = 0
        months = 0
        list_to_detection = str(information).split()
        
        for word in list_to_detection:
            
            if word.isdigit():
                int_list.append(int(word))
        
        
        if len(int_list)>0:
            years = int_list[0]
                       
            if len(int_list)>1 and int_list[1] <= 12:
                months = int_list[1]
                
        return int((years*12+months))

data['Опыт работы (месяц)'] = data['Опыт работы'].apply(experience_detection)
data = data.drop(['Опыт работы'], axis = 1)

4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
#ваш код здесь
million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 
                            'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 
                            'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']

def splitter(enterinf):
    list_for_work = enterinf.split(sep = ',')
    
    if len(list_for_work)>3:
        del list_for_work[1]
            
    return list_for_work


def to_city(information):
        
    city = splitter(information)[0].strip(' ')
        
    if city in million_cities:
        return 'город-миллионник'
        
    elif city == 'Москва' or city == 'Санкт-Петербург':
        return city
        
    else:
        return 'другие'
    
def moving(information):
    enable = splitter(information)[1].split()
    
    
    if enable[0].lower() == 'не':
        return False
    
    else: return True
    
    
def business_trip(information):
    enable = splitter(information)[2].split()
    
    if enable[0].lower() == 'не':
        return False
    
    else: return True

data['Город'] = data['Город, переезд, командировки'].apply(to_city)
data['Готовность к переездам'] = data['Город, переезд, командировки'].apply(moving)
data['Готовность к командировкам'] = data['Город, переезд, командировки'].apply(business_trip)
data = data.drop(['Город, переезд, командировки'], axis = 1)

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
#ваш код здесь
emp_list = [
    'полная занятость', 'частичная занятость',
    'проектная работа', 'стажировка', 'волонтерство'
]
graph_list = [
    'полный день', 'смешанный график', 'гибкий график', 
    'удаленная работа', 'вахтовый метод'
]

def to_bool(information, cat):
    splitted_list = information.split(',')
    
    for i in splitted_list:
        i = i.strip(' ')
        
        if i == cat: return True
        
    return False
        

for i in emp_list:
    data[i] = data['Занятость'].apply(to_bool, cat = i)
    
for n in graph_list:
    data[n] = data['График'].apply(to_bool, cat = i)
    
data = data.drop(['Занятость', 'График'], axis = 1)


In [ ]:
data.isnull().sum()

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
#ваш код здесь
currency_data = pd.read_csv('/home/gor/Загрузки/ExchangeRates.zip', compression='zip')
currency_for_work = currency_data.copy()
data['Обновление резюме'] = pd.to_datetime(data['Обновление резюме']).dt.date
currency_for_work['date'] = pd.to_datetime(currency_for_work['date'])
currency_for_work = currency_for_work.drop(['per', 'vol', 'time'], axis=1)
data['Размер ЗП']=data['ЗП'].apply(lambda x: int(x.split()[0]))
data['Валюта'] = data['ЗП'].apply(lambda x: x.split()[1].strip(' '))
data = data.drop(['ЗП'], axis = 1)
data['Обновление резюме'] = pd.to_datetime(data['Обновление резюме'])


def to_right_cur(cur):
    
    if cur == 'бел.руб.': return 'BYN'
    
    elif cur == 'грн.': return 'UAH'
    
    elif cur == 'сум': return 'UZS'
    
    else: return cur
    
    
data['Валюта'] = data['Валюта'].apply(to_right_cur)
joined = data.merge(currency_for_work, how='left', left_on=['Валюта', 'Обновление резюме'], right_on=['currency','date'])
values = {
         'close' : 1,
         'proportion' : 1
         }
joined = joined.fillna(values)
joined['Размер ЗП'] = joined['close']/joined['proportion']*joined['Размер ЗП']
joined = joined.drop(['currency','date','close','proportion'], axis=1)

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# ваш код здесь
import plotly.offline as py
py.init_notebook_mode(connected = True)


data = joined.copy()

fig = px.histogram(data_frame=data, 
                   x='Возраст',
                   title='Age of employees',
                   marginal='box')
fig.show()

На графике мы можем наблюдать, что основная часть соискателей в возрасте от 20 до 55 лет. График имеет правостороннюю ассиметрию. Максимальный пик находится на отметке 30 лет.

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# ваш код 

fig = px.histogram(data_frame=data, 
                   x='Опыт работы (месяц)',
                   title='Expierence of employees',
                   marginal='box')
fig.show()


Мода, согласно графика, находится на уровне 82 месяцев работы. Опыт работы большинства соискателей находится в промежутке от 60 до 154. Есть одна отчетливо определяемая аномалия, которая находится на уровне 1188 месяцев, она соответствует соискателю возрастом 100 лет (может, реально есть такой). 

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:


fig = px.histogram(data_frame=data, 
                   x='Размер ЗП',
                   title='Employees salary',
                   marginal='box')
fig.show('notebook')


Предельные значения заработной платы - от 1 до 24,3 млн рублей. Заработная плата большинства соискателей находится в интервале от 37 до 95 тысяч рублей. Прямой взаимосвязи с возрастом в графике не наблюдаю.

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
salary_per_education = data.groupby(by='Образование', as_index=True)['Размер ЗП'].mean()

sal_ed_fig = px.bar(data_frame=salary_per_education, title='Salary per education level')
sal_ed_fig.show()

Наибольший уровень заработной платы соответствует соискателям, имеющим высшее образование, наименьший - среднее специальное. Считаю, что такой признак, как образование, играет основополагющую роль в формировании запросов на заработную плату, так как человек, имеющий высшее образование однозначно захочет за вложенный в себя, в свое образование, труд получить достойную оплату. Но это не единственный показатель. 

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
sal_per_city = data[data['Размер ЗП']<1e6].groupby('Город', as_index=True)['Размер ЗП'].mean()

sal_city_fig = px.bar(data_frame=sal_per_city, title='Salary per city')
sal_city_fig.show('notebook')

Как видно из графика, наиболее высокие зарплаты наблюдаются в городах федерального значения, это связано с более высоким уровнем жизни в указанных городах. Наименьший показатель - в городах-миллионниках. Чуть больше - в остальных городах России, что, скорее всего, связано с наличием в подобных городах предприятий, занимающихся добычей полезных ископаемых, а также в связи с высокими зарплатами в северной части страны.
Данный признак является важным для формирования заработной платы, что мы можем наблюдать из графика.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
sal_move = data.groupby(by=['Готовность к переездам', 'Готовность к командировкам'], as_index=False)['Размер ЗП'].median()
sal_move_bar = px.bar(
    data_frame = sal_move,
    x='Готовность к переездам',
    y='Размер ЗП',
    barmode='group',
    color='Готовность к командировкам',
    title='Salary per moving and errands'
    )
sal_move_bar.show()

Как мы можем увидеть из графика, соискатели, готовые к пеерздам и командировкам, предъявляют более высокие требования к заработной плате в сранении с остальными категориями. Чуть меньше требуемый увроен заработной платы, кто не готов к переездам, но готов к командировкам. Далее идут соискатели, готовые к переезду, но не готовые к командировкам. И самый низкий уровень требуемой заработной платы у тех, кто не готов ни к командировкам ни к переездам. Таким образом, главенствующим фактором из рассматриваемых является готовность к командировкам, готовность к переездам, в свою очередь, оказывает меньшее влияние на заработную плату соискателей.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
pivot_sal_age_ed = pd.pivot_table(
                                data=data,
                                values='Размер ЗП',
                                index='Возраст',
                                columns='Образование',
                                aggfunc='mean'
                                )
sal_age_ed_fig = plt.figure(figsize=(10,6))
sal_age_heatmap = sns.heatmap(data=pivot_sal_age_ed)
sal_age_heatmap.set_title('Salary per age and education level')

Из тепловой карты мы можем увидеть, что наибольший уровень зарплат требуют соискатели, имебщие высшее образование в возрасте от 37 до 48 (косвенный признак опытного сотрудника), далее идет группа с неоконченным высшим образованием в том же возрастном диапазоне. Категория соискателей, имеющих среднее обрзование заметно моложе (возрастной предел в районе 58-60 лет). Между категориями соискателей, имеющих неоконченное и среднее не зависимо от возраста, наблюдается большая просадка в размере требуемой заработной платы. Наименьшую заработную плату просят соискатели со средним профессиональным образованием.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
data['Опыт работы (год)'] = data['Опыт работы (месяц)']//12
exp_per_age = plt.figure(figsize=(10,6))
exp_per_age_fig = sns.scatterplot(
                                  data=data,
                                  x = 'Возраст',
                                  y = 'Опыт работы (год)',
                                  ci='sd'
                                  )
exp_per_age_fig.set_title('Expierence per age')

In [ ]:
exp_fig = px.scatter(
                    data_frame=data,
                    x='Возраст',
                    y='Опыт работы (год)',
                    range_x=[0,100],
                    range_y=[0,100]
)
exp_fig.add_shape(
                  type='line',
                  yref='y',
                  xref='x',
                  x0=0,
                  y0=0,
                  x1=100,
                  y1=100,
                  line=dict(color='black', width=1)
                 )
exp_fig.show('notebook')

Из построенного графика видно, что имеется всего шесть аномалий. Седьмая аномалия, если ее можно считать таковой находится в точке [100, 2].

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
auto_city_sal = pd.pivot_table(
                               data=data,
                               values='Размер ЗП',
                               index='Город',
                               columns='Авто',
                               aggfunc='mean'
                               )
acs_fig = plt.figure(figsize=(10,6))
acs_heatmap = sns.heatmap(data=auto_city_sal)
acs_heatmap.set_title('Salary per car and city')

Как видно из графика, существует взаимосвязь между городом, в котором проживает соискатель, наличия у него автомобиля (условно предполагаем, что если авто не указано, то его просто нет). Так, в Москве при наличии автомобиля требуемый уровень заработной платы самый высокий, в случае же его отсутсвия - много ниже. Подобный вывод можно сделать в отношении всех городов: при наличии автомобиля требуемый уровень заработной платы выше независимо от города.

In [ ]:
sal_per_exp = data.groupby(by='Опыт работы (год)', as_index=True)['Размер ЗП'].mean()
sal_per_exp_fig = px.bar(data_frame=sal_per_exp, title='Salary per experience')
sal_per_exp_fig.show()

Из графика можно сделать следующий вывод: размер требуемой заработной платы прямой зависимости от опыта работы соискателя не имеет. Пик наблюдается в промежутке от 15 до 26 лет работы. Считаю, что это явление более социальное, так как люди с опытом работы в указанном диапазоне (с учетом того, что они начали работать в возрасте 18-23 лет) имеют достаточный опыт для требования более высокой заработной платы по сравнению с людьми, проработавшими более 35 лет и обладающими "советской закалкой" и более "приземленными" взглядами на жизнь.
Что касается категории соискателей с опытом работы менее 16 лет, то тут можно предположить, что рассматриваемая категория ввиду меньшего опыта требует меньший уровень заработной платы. 
Также на графике видны выбросы на отметке с опытом работы 51 год, где средний требуемый уровень заработной платы составляет 350 тысяч рублей. А также отметка с опытом работы 100 лет (однозначная аномалия).

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
dupl_columns = list(data.columns.unique())

mask = data.duplicated(subset=dupl_columns)
data_duplicates = data[mask]
print(f'Duplicates number is {data_duplicates.shape[0]}')


2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
data.shape[0]

In [ ]:
subset = ['Последнее/нынешнее место работы','Последняя/нынешняя должность']
data = data.dropna(
           axis=0,
           subset=subset
           )
data = data.fillna({
                    'Опыт работы (месяц)' : data['Опыт работы (месяц)'].median(),
                    'Опыт работы (год)' : data['Опыт работы (год)'].median()/12
                    })
data['Опыт работы (месяц)'].mean()

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
# ваш код здесь
drop_ind = list((data[data['Размер ЗП']>1e6]).index.unique())+list((data[data['Размер ЗП']<1e3]).index.unique())
data = data.drop(drop_ind)

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
# ваш код здесь
drop_ind = list((data[data['Возраст']<=data['Опыт работы (год)']]).index)
data = data.drop(drop_ind)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
# ваш код здесь
fig,axes = plt.subplots(1,1, figsize=(12,8))
log_age = np.log(data['Возраст']+1)
histplot = sns.histplot(log_age, bins=30)
histplot.set_title('Age of employees')
histplot.axvline(log_age.mean(), color='k', lw=2)

In [37]:
def outliers_z_score(data, feature, log_scale=False, left=3, right=3):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers,cleaned = outliers_z_score(data, 'Возраст', log_scale=True, right=4)

,Ищет работу на должность:,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,Образование,Пол,Возраст,Опыт работы (месяц),...,стажировка,волонтерство,полный день,смешанный график,гибкий график,удаленная работа,вахтовый метод,Размер ЗП,Валюта,Опыт работы (год)
0,Системный администратор,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,2019-04-16,Имеется собственный автомобиль,неоконченное высшее,М,39,202.0,...,False,False,False,False,False,False,False,29000.0,руб.,16.0
1,Технический писатель,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,2019-12-04,Не указано,высшее,М,60,233.0,...,False,False,False,False,False,False,False,40000.0,руб.,19.0
2,Оператор,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,2019-04-16,Не указано,среднее специальное,Ж,36,123.0,...,False,False,False,False,False,False,False,20000.0,руб.,10.0
3,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,2019-08-04,Не указано,высшее,М,38,225.0,...,False,False,False,False,False,False,False,100000.0,руб.,18.0
4,Региональный менеджер по продажам,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,2019-04-22,Не указано,высшее,Ж,26,67.0,...,False,False,False,False,False,False,False,140000.0,руб.,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44739,"Финансист, аналитик, экономист, бухгалтер, мен...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,Высшее образование 2015 Московский гуманитарн...,2019-04-22,Не указано,высшее,М,30,91.0,...,False,False,False,False,False,False,False,50000.0,руб.,7.0
44740,"Системный администратор, IT-специалист",ИП Пестрецов,Предприниматель,Высшее образование (Бакалавр) 2016 Воронежски...,2019-04-22,Не указано,высшее,М,27,84.0,...,False,False,False,False,False,False,False,39000.0,руб.,7.0
44741,"Аналитик данных, Математик","ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,Высшее образование 2000 Южно-Уральский госуда...,2019-09-04,Не указано,высшее,Ж,48,257.0,...,False,False,False,False,False,False,False,40000.0,руб.,21.0
44742,Контент-менеджер,IQ-Maxima,Менеджер проектов,Высшее образование 2015 Тамбовский государств...,2019-04-26,Имеется собственный автомобиль,высшее,М,24,46.0,...,False,False,False,False,False,False,False,20000.0,руб.,3.0


In [35]:
cleaned.to_csv('cleaned_HH_proj.csv')

График имеет правостороннюю ассиметрию. В категорию выбросов попали 2 соискателя возрастом 15 лет, а также 1 соискатель возрастом 100 лет.